# JRC Sample Extraction
<div>
<img src="jrc_ec_logo.jpg" width="400"/>
</div>

Solution that selects inspection samples for the quality assessments (QA) of the Area Monitoring System (AMS) and GeoSpatial Application (GSA) 

[INTRODUCTION + GENERAL INSTRUCTIONS HERE]

Authors: Fernando Fahl (fernando.fahl@ext.ec.europa.eu), Mateusz Dobrychłop (mateusz.dobrychlop@ext.ec.europa.eu), Ferdinando Urbano (ferdinando.urbano@ec.europa.eu)

## Load parcel list

Run the code cell below to display a set of widgets that will allow you to select your input file.

[INSTRUCTIONS + PARCEL FILE FORMATTING HERE]

In [1]:
import sample_extraction_gui as gui
uploaded_files = gui.display_parcel_input_config()

# input/MT_parcels_new.csv

## Define bucket targets

Run the code cell below to display a set of widgets that will allow you to define ua group bucket targets.

[INSTRUCTIONS + TARGET FILE FORMATTING HERE]

In [2]:
gui.display_bucket_targets_config()

# what if the parcels are not loaded, then this cell is loaded (no widgets are shown), and then the parcel is loaded?
# add reload button? maybe it could be shown when the "no ua_grp_id" error is shown?

## Set output

## Set optional parameters

To be added in future versions.

In [ ]:
# gui.display_advanced_config()

## Running extraction

In [3]:
import sample_extraction

buckets = sample_extraction.prepare_buckets(gui.PARAMETERS["ua_groups"])
#mapping = sample_extraction.create_ua_grp_id_map(buckets)
#buckets = sample_extraction.translate_buckets(buckets, mapping)

parcels = sample_extraction.prepare_input_dataframe(gui.PARAMETERS["parcels_df"])
#parcels = sample_extraction.translate_parcel_df(parcels, mapping)


print(parcels.head())
print(buckets)

sample_extraction.iterate_over_interventions(parcels, buckets) #, pbars)    

# Indicate the reason why the run ended
if sample_extraction.buckets_full(buckets):
    print("\nAll buckets full!")
else:
    print("\nSome buckets not full!")

sample_extraction.generate_output(buckets)   
print("\nOutput file generated.")

        gsa_par_id  gsa_hol_id  ua_grp_id  ranking  covered  \
0  ZTN0-5668-00153        3731          0        1        1   
1  ZTN0-5668-00153        3731          1        1        1   
2  ATD0-4671-00080        9377          2        2        1   
3  ATD0-4671-00080        9377          1        2        1   
4  RBT0-4470-00646        8601          1        3        1   

                     row_id  order_added  
0   ZTN0-5668-00153_3731_E5            0  
1  ZTN0-5668-00153_3731_ANC            0  
2  ATD0-4671-00080_9377_BIS            0  
3  ATD0-4671-00080_9377_ANC            0  
4  RBT0-4470-00646_8601_ANC            0  
{0: {'target': 250, 'parcels': []}, 1: {'target': 300, 'parcels': []}, 2: {'target': 300, 'parcels': []}, 3: {'target': 24, 'parcels': []}, 4: {'target': 50, 'parcels': []}, 5: {'target': 140, 'parcels': []}, 6: {'target': 300, 'parcels': []}, 7: {'target': 70, 'parcels': []}, 8: {'target': 14, 'parcels': []}, 9: {'target': 3, 'parcels': []}, 10: {'target': 4, 

main changes:
- when looping through a holding, every time a parcel is checked, all interventions from it have to be added to all possible buckets (unless bucket is full)
- when looping through holding parcels, stop adding parcels to a bucket if 3 parcels from that holding are already in the bucket. but the other buckets still have to be checked. implement some kind of local bucket list? that is then merged with the big one?

- work on the interface a bit
- add some parameters


---
- 3 interventions per bucket


- parcel -> check all interventions

- holding check -> 



- once a bucket is filled, maybe remove all rows related to it from the main list?
---


modifiable parameters: 
- 3% limit
- just the parcels covered by area images or whole list?
- priority to the covered?
- 

- costas, augusta, slavko, gilbert, paulo




- 3% rule = for all buckets, if 3% of holdings are added, stop adding to a bucket even if below threshold. if you reach 3% of holdings, only keep adding parcels from the 3% holdings already added
- grouping rows together (1 row per parcel with a list of intervensions in one columns)